## Data parameters

In [ ]:
#use these parameters for testing this notebook outside of the automated loop of q1k_automated_reports.ipynb
#subject_id_in = "10086_F1"
#subject_id_out = "10086F1"
#task_id_in = "AS"
#task_id_in_et = "AS" 
#task_id_out = "AS"
#run_id = "1"
#session_id = "01"
#project_path = "/project/def-emayada/q1k/experimental/hsj/"
#dataset_group = "experimental"

#use these empty parameters when executing this notebook from an automation script.
subject_id_in = ""
subject_id_out = ""
task_id_in = ""
task_id_in_et = "" 
task_id_out = ""
run_id = ""
session_id = ""
project_path = ""
dataset_group = ""

print(dataset_group)


In [ ]:
# Create the task specific event string list
if task_id_out == 'AEP':
    event_dict_offset = 1
    din_str = ('DIN4','DIN5')
if task_id_out == 'AS':
    event_dict_offset = 1
    din_str = ('DIN2','DIN2')
if task_id_out == 'GO':
    event_dict_offset = 1
    din_str = ('DIN2','DIN3')
if task_id_out == 'TO':
    event_dict_offset = 1
    din_str = ('DIN4','DIN5')
if task_id_out == 'VEP':
    event_dict_offset = 1
    din_str = ('DIN2','DIN3')
    
print(dataset_group)

In [ ]:
# import packages
import pandas as pd
import numpy as np
import mne
import mne_bids
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as py
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"
import q1k_init_tools as qit
import glob
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns

In [ ]:
# define the function for generating the input path and file name
def generate_session_ids (dataset_group, path, task, subject_id_in,  run_id):
    if dataset_group == "control": 
        #session_path_eeg = project_path + 'sourcefiles/' + subject_id + '_' + session_id[1] + '/' + subject_id + '_' + session_id[1] + '_eeg/'
        session_path_eeg = project_path + 'sourcefiles/' + subject_id_in + '/' + subject_id_in + '_eeg/'
        #session_file_name_eeg = glob.glob(session_path_eeg + '*_' + task_id_in + '_*.mff')
        session_file_name_eeg = glob.glob(session_path_eeg + '*_' + task_id_in + '_*.mff')

        #session_path_et = project_path + 'sourcefiles/' + subject_id + '_' + session_id[1] + '/' + subject_id + '_' + session_id[1] + '_et/'
        #session_file_name_et = glob.glob(session_path_et + '*_' + task_id_in_et + '_*.asc')

    elif dataset_group == "experimental":
        #session_path_eeg = project_path + '/sourcedata/eeg/Q1K_HSJ_' + subject_id + '_' + session_id +'/'
        session_path_eeg = project_path + '/sourcedata/eeg/Q1K_HSJ_' + subject_id_in + '/'
        session_file_name_eeg = glob.glob(session_path_eeg + '*' + task_id_in + '*.mff')
        print(session_path_eeg)
        print(session_file_name_eeg)

        #session_path_et = project_path + 'sourcedata/' + subject_id + '_' + session_id[1] + '/' + subject_id + '_'  + '_et/'
        session_path_et = project_path + 'sourcedata/' + subject_id_in + '/' + subject_id_in + '_'  + '_et/'
        session_file_name_et = glob.glob(session_path_et + '*_' + task_id_in_et + '_*.asc')
    return session_file_name_eeg, session_file_name_et

In [ ]:
subject_id_in

In [ ]:
# generate the input paths
session_file_name_eeg, session_file_name_et = generate_session_ids(dataset_group, project_path, task_id_in, subject_id_in, run_id)

In [ ]:
# read the input EEG session
print('Reading: ' + session_file_name_eeg[0])

# Read the EEG data
raw = mne.io.read_raw_egi(session_file_name_eeg[0])

In [ ]:
# create the EEG event structures
eeg_events = mne.find_events(raw, shortest_event = 1)
#eeg_events = mne.find_events(raw, stim_channel="STI 014", shortest_event = 1)

eeg_event_dict = qit.get_event_dict(raw, eeg_events, event_dict_offset)

In [ ]:
eeg_event_dict

In [ ]:
# this is a patch that will become obsolete... accounting for missing DIN types in the recording..
exists_in_dict = [din in eeg_event_dict for din in din_str]
print(din_str)
print(exists_in_dict)
if all(exists_in_dict):
    print("Both strings exist in eeg_event_dict.")
elif any(exists_in_dict):
    existing_string = din_str[exists_in_dict.index(True)]
    din_str = (existing_string, existing_string)
    print(f"Only one string exists. din_str updated to: {din_str}")
else:
    print("Neither string exists in eeg_event_dict.")
print(din_str)

In [ ]:
# peak... at the EEG event scatter plot.. event time stamp by label index
fig=px.scatter(x=eeg_events[:,0],y=eeg_events[:,2])
fig.update_layout(title='Original event times')
fig.update_xaxes(title_text='Time of event(ms)')
fig.update_yaxes(title_text='Event index')
py.iplot(fig)


In [ ]:
# print the scatterplot to an html file for easy exploration.
fig.write_html("event_test.html")

In [ ]:
# get the distance between DIN events of interest
din_diffs = []
din_diffs_time = []
last_din = 0
# Iterate over the array
for row in eeg_events:
    # Check if the third column corresponds to e1 or e2 values
    if row[2] == eeg_event_dict[din_str[0]] or row[2] == eeg_event_dict[din_str[0]]:
        # Calculate the sequential difference between values in the first column
        if last_din > 0:
            din_diffs.append(row[0] - last_din)
            din_diffs_time.append(row[0])
            last_din = row[0]
        else:
            last_din = row[0]
            #print(last_din)


In [ ]:
# peak... at distances between DIN events of interest
fig=px.scatter(x=din_diffs_time, y=din_diffs)
fig.update_layout(title='Time between DIN events')
fig.update_xaxes(title_text='Time of event(ms)')
fig.update_yaxes(title_text='Inter event interval')

In [ ]:
# handle task specific EEG event interpretation..
eeg_events, eeg_stims, eeg_iti, eeg_din_offset, eeg_event_dict, new_events = qit.eeg_event_test(eeg_events, eeg_event_dict, din_str, task_name=task_id_out)

In [ ]:
# print some stim DIN event summaries
column_values = eeg_events[:, 2]

# Get unique values and their counts
unique_values, counts = np.unique(column_values, return_counts=True)

# Print the counts of each unique value
print("Counts of each event ID value:")
for value, count in zip(unique_values, counts):
    print(f"Event ID: {value}, Count: {count}")

# Print the event labels
print("Label\tValue")
for label, value in eeg_event_dict.items():
    print(f"{label}\t{value}")

In [ ]:
# peak... at the EEG event scatter plot.. event time stamp by label index... including new *_d DIN events
fig=px.scatter(x=eeg_events[:,0],y=eeg_events[:,2])
fig.update_layout(title='DIN updated event times')
fig.update_xaxes(title_text='Time of event(ms)')
fig.update_yaxes(title_text='Event index')

fig.show()

In [ ]:
# peak... at the distance between *_d stim DIN events
fig=px.scatter(x=eeg_stims[1:,0],y=eeg_iti)
fig.update_layout(title='Stim DIN event Inter Trial Intervals (ITI)')
fig.update_xaxes(title_text='Time of event(ms)')
fig.update_yaxes(title_text='Stim DIBN event ITI (ms)')
fig.show()

In [ ]:
# peak... at the distance between stim events and *_d stim DIN events
fig=px.scatter(x=eeg_stims[:,0],y=eeg_din_offset)
fig.update_layout(title='Stim DIN offsets')
fig.update_xaxes(title_text='Time of event(ms)')
fig.update_yaxes(title_text='Stim DIN offset (ms)')

fig.show()

In [ ]:
# show the snsor topography
fig = raw.plot_sensors(show_names=True)

In [ ]:
# print the updated event dictionary
eeg_event_dict

In [ ]:
# write the BIDS output files
# specify power line frequency as required by BIDS
raw.info["line_freq"] = 60
raw.info['device_info']['type'] = raw.info['device_info']['type'].replace(' ', '-')

bids_path = mne_bids.BIDSPath(
    subject=subject_id_out, session=session_id, task=task_id_out, run="1", datatype="eeg", root=project_path
)

print(bids_path)
mne_bids.write_raw_bids(
    raw=raw,
    bids_path=bids_path,
    events=eeg_events,
    event_id=eeg_event_dict,
    format = "EDF",
    overwrite=True,
)

In [ ]:
# write the last saved verion of this notebook
#!jupyter nbconvert --output {"./../session_reports/" + bids_path.basename + ".html"} --no-input --to html q1k_eeget_init.ipynb
#!jupyter nbconvert --output {"./../session_reports/"  + task_id_in + "/" + subject_id + ".html"} --TagRemovePreprocessor.remove_all_outputs_tags='{'exclude'}' --no-input --to html q1k_generate_individual_reports.ipynb
#f'Total of {len(outlier_accum)} outliers out of {all_accounts.size} accounts'